In [ ]:
def forward(V, trans, emiss, pi):
    """Given a sequence of Visible states (v-emitted bases) calculate the the probability that the HMM will be in a
        particular hidden state (s-AT/CG rich) at a particular time step t
        alpha_j(t) = p(v_k(1),…,v_k(t),s(t)=j)
               
        V = sequence of visible states {v(1), ..., v(t)}
        trans = transition probability matrix
        emiss = emission probability matrix
        pi = initial distribution
        
        alpha_j(t) = pi_j*e_jk when t=1
        alpha_j(t) = e_jk*(alpha_i(t-1)*t_ij) sum over i for i=1 to i=M when t >= 1"""
    
    alpha = np.zeros((V.shape[0], trans.shape[0])) # initialising alpha matrix containing the prob of each hidden state for
    #each time step alpha.T = {[alpha_AT(0), alpha_CG(0)], ..., [alpha_AT(T), alpha_CG(T)]} 
    
    if(V[0] == 'A'):
        alpha[0,:] = pi*emiss[:,0] # [alpha_AT(0)=p(s=AT_rich)*p(v=A|s=AT_rich), alpha_CG(0)=p(s=CG_rich)*p(v=A|s=CG_rich)]
    elif(V[0] == 'T'):
        alpha[0,:] = pi*emiss[:,1] # [alpha_AT(0)=p(s=AT_rich)*p(v=T|s=AT_rich), alpha_CG(0)=p(s=CG_rich)*p(v=T|s=CG_rich)]
    elif(V[0] == 'C'):
        alpha[0,:] = pi*emiss[:,2] # [alpha_AT(0)=p(s=AT_rich)*p(v=C|s=AT_rich), alpha_CG(0)=p(s=CG_rich)*p(v=C|s=CG_rich)]
    else:
        alpha[0,:] = pi*emiss[:,3] # [alpha_AT(0)=p(s=AT_rich)*p(v=G|s=AT_rich), alpha_CG(0)=p(s=CG_rich)*p(v=G|s=CG_rich)]
    
    for t in range(1, V.shape[0]):
        for j in range(trans.shape[0]):
            if(V[0] == 'A'):
                alpha[t,j] = alpha[t-1].dot(trans[:,j])*emiss[j,0]
            elif(V[0] == 'T'):
                alpha[t,j] = alpha[t-1].dot(trans[:,j])*emiss[j,1] 
            elif(V[0] == 'C'):
                alpha[t,j] = alpha[t-1].dot(trans[:,j])*emiss[j,2] 
            else:
                alpha[t,j] = alpha[t-1].dot(trans[:,j])*emiss[j,3] 
 

    return alpha

In [ ]:
def backward(V, trans, emiss):
    """ find the probability that the machine will be in hidden state s_i at time step t and will generate the remaining
    part of the sequence of the visible symbol
    beta_i(t) = p(v_k(t+1),...,v_k(T)|s(t)=i)
    
    V = sequence of visible states {v(1), ..., v(t)}
    trans = transition probability matrix
    emiss = emission probability matrix
    
    beta_i(t) = 1 when t=T
    beta_i(t) = t_ij*e_jk*beta_j(t+1) sum over j for j=0 to j=M when t<T"""
    
    beta = np.zeros((V.shape[0], trans.shape[0])) # initialising beta matrix containing the joint prob of the set of
    #visible states {v_k(t+1),...,v_k(T)} conditioned upon the hidden state s(t)
    #each time step beta.T = {[beta_AT(t+1), beta_CG(t+1)], ..., [beta_AT(T), beta_CG(T)]}
    
    beta[V.shape[0]-1] = np.ones((trans.shape[0])) # setting [beta_AT(T)=1, beta_CG=1]
    
    # Loop in backward way from T-1 to
    # Due to python indexing the actual loop will be T-2 to 0
    for t in range(V.shape[0]-2,-1,-1): #range([start], stop[, step])
        for j in range(trans.shape[0]):
            if(V[t+1] == 'A'):
                beta[t,j] = trans[j,:].dot(emiss[:,0]*beta[t+1])
                #beta[t,j] = (beta[t+1]*emiss[:, V[t + 1]]).dot(trans[j,:])
            elif(V[t+1] == 'T'):
                beta[t,j] = trans[j,:].dot(emiss[:,1]*beta[t+1])
            elif(V[t+1] == 'C'):
                beta[t,j] = trans[j,:].dot(emiss[:,2]*beta[t+1])
            else:
                beta[t,j] = trans[j,:].dot(emiss[:,3]*beta[t+1])
                
    return beta